In [1]:
import sys
import numpy as np
import random as rn
import pandas as pd
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
# from torchnlp.datasets import imdb_dataset      # --> We are using our own uploaded dataset.
from pytorch_pretrained_bert import BertTokenizer
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import clear_output
from transformers import AutoTokenizer, AutoModelForMaskedLM
import matplotlib.pyplot as plt


import sqlite3
import jieba
import jieba.posseg as pseg

In [2]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

In [3]:
import pathlib
pathlib.Path().resolve()

WindowsPath('C:/Users/User/Downloads/Gary/Therapy-Chatbot-Deploying-NLP/Training')

In [4]:
train_data = pd.read_csv('../train.csv')
test_data = pd.read_csv('../test.csv')

In [5]:
train_data = train_data.to_dict(orient='records')
test_data = test_data.to_dict(orient='records')
len(train_data), len(test_data)

(20626, 6873)

In [6]:
train_texts, train_labels = list(zip(*map(lambda d: (d['title'], d['label']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['title'], d['label']), test_data)))
type(train_labels)
len(train_texts), len(train_labels), len(test_texts), len(test_labels)
train_labels = [x-1 for x in train_labels]
test_labels = [x-1 for x in test_labels]

In [7]:
def jiebaSlice(content,mode):
    stopword_set = []
    content = str(content)
    with open('../Analyzing/stopword.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.append(stopword.strip('\n'))
    
    content = content.strip('\n')
    if mode == "POSSEG":
        words = pseg.cut(content,use_paddle=True)
        slicedWords = []
        for word, flag in words:
            if word not in stopword_set:
                slicedWords.append(word)
        return slicedWords
    elif mode == "CUT_HMM":
        seg_list = jieba.cut(content,HMM=True,cut_all=True)
        slicedWords = list(seg_list)
        return slicedWords
    elif mode == "CUT_FOR_SEARCH":
        seg_list = jieba.cut_for_search(content,HMM=True)
        slicedWords = list(seg_list)
        return slicedWords
    elif mode == "NORMAL":
        seg_list = jieba.cut_for_search(content)
        slicedWords = list(seg_list)
        return slicedWords  

In [8]:
jiebaSlice(train_texts[3],'CUT_FOR_SEARCH')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.500 seconds.
Prefix dict has been built successfully.


['為', '什麼', '要', '上', '大學', '了', '卻', '還沒交過', '男友', '😢']

In [9]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

In [10]:
train_tokens = list(map(lambda t: ['[CLS]'] + jiebaSlice(t,'CUT_FOR_SEARCH') + ['[SEP]'], train_texts))

test_tokens = list(map(lambda t: ['[CLS]'] + jiebaSlice(t,'CUT_FOR_SEARCH') + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)

(20626, 6873)

In [11]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((20626, 512), (6873, 512))

In [12]:
print(train_tokens[30])

['[CLS]', '民進', '黨', '政府', '真的', '是', '爛到', '無', '可救', '可救藥', '[SEP]']


In [13]:
print(train_tokens_ids[30])

[ 101  100 7955  100  100 3221  100 4192  100  100  102    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [14]:
train_y = np.array(train_labels)
test_y = np.array(test_labels) 
train_y.shape, test_y.shape

((20626,), (6873,))

In [15]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [17]:
# baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LogisticRegression()).fit_transform(train_texts, train_labels)

In [18]:
# baseline_predicted = baseline_model.predict(test_texts)

In [19]:
# print(classification_report(test_labels, baseline_predicted))

In [20]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 10)
#         self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
#         proba = self.sigmoid(linear_output)
        return F.log_softmax(linear_output)

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [23]:
bert_clf = BertBinaryClassifier()
# bert_clf = Net()
bert_clf = bert_clf.cuda()

In [24]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'409.732608M'

In [25]:
x = torch.tensor(train_tokens_ids[:3]).to(device)
y, pooled = bert_clf.bert(x, output_all_encoded_layers=False)
x.shape, y.shape, pooled.shape

(torch.Size([3, 512]), torch.Size([3, 512, 768]), torch.Size([3, 768]))

In [26]:
y = bert_clf(x)
y.cpu().detach().numpy()

C:\Users\User\AppData\Local\Temp\ipykernel_17044\1901059768.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(linear_output)


array([[-3.375049 , -3.1389227, -1.1008068, -2.4563272, -2.2747593,
        -2.5494456, -3.4086561, -2.6647897, -2.0225224, -2.428889 ],
       [-3.6160562, -2.580394 , -1.0316668, -2.5483437, -2.1665225,
        -3.0281823, -3.426412 , -2.8819594, -1.9583983, -2.657085 ],
       [-3.3498976, -2.9017181, -1.5290862, -2.214488 , -2.4176364,
        -2.6784186, -2.9560342, -2.3406365, -1.7454438, -2.2693636]],
      dtype=float32)

In [27]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'5762.029056M'

In [28]:
y, x, pooled = None, None, None
torch.cuda.empty_cache()     # Clearing Cache space for fresh Model run
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'409.732608M'

In [29]:
BATCH_SIZE = 1
EPOCHS = 30

In [30]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'409.732608M'

In [31]:
print(train_tokens_tensor.shape)
print(train_masks_tensor.shape)
print(train_y_tensor.shape)
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

torch.Size([20626, 512])
torch.Size([20626, 512])
torch.Size([20626, 1])


In [33]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [34]:
torch.cuda.empty_cache()

In [35]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [36]:
epoch_loss = []
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        labels = torch.squeeze(labels,1)
        labels = labels.to(torch.long)
        print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)

        logits = bert_clf(token_ids, masks)
        
#         loss_func = nn.BCELoss()
        loss_func = nn.CrossEntropyLoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        bert_clf.zero_grad()
        batch_loss.backward()
        
        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))
    epoch_loss.append(train_loss)

Epoch:  10
20625/20626.0 loss: 1.6549073162537329 


In [37]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        labels = torch.squeeze(labels,1)
        labels = labels.to(torch.long)
        logits = bert_clf(token_ids, masks)
#         loss_func = nn.BCELoss()
        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        print("\r" + "{0}/{1}".format(step_num, len(test_data) / BATCH_SIZE))
        bert_predicted.append(list(numpy_logits[0, :]).index(max(list(numpy_logits[0, :]))))
        all_logits += list(numpy_logits[:, 0])

0/6873.0
1/6873.0
2/6873.0
3/6873.0
4/6873.0
5/6873.0
6/6873.0


C:\Users\User\AppData\Local\Temp\ipykernel_17044\1901059768.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(linear_output)


7/6873.0
8/6873.0
9/6873.0
10/6873.0
11/6873.0
12/6873.0
13/6873.0
14/6873.0
15/6873.0
16/6873.0
17/6873.0
18/6873.0
19/6873.0
20/6873.0
21/6873.0
22/6873.0
23/6873.0
24/6873.0
25/6873.0
26/6873.0
27/6873.0
28/6873.0
29/6873.0
30/6873.0
31/6873.0
32/6873.0
33/6873.0
34/6873.0
35/6873.0
36/6873.0
37/6873.0
38/6873.0
39/6873.0
40/6873.0
41/6873.0
42/6873.0
43/6873.0
44/6873.0
45/6873.0
46/6873.0
47/6873.0
48/6873.0
49/6873.0
50/6873.0
51/6873.0
52/6873.0
53/6873.0
54/6873.0
55/6873.0
56/6873.0
57/6873.0
58/6873.0
59/6873.0
60/6873.0
61/6873.0
62/6873.0
63/6873.0
64/6873.0
65/6873.0
66/6873.0
67/6873.0
68/6873.0
69/6873.0
70/6873.0
71/6873.0
72/6873.0
73/6873.0
74/6873.0
75/6873.0
76/6873.0
77/6873.0
78/6873.0
79/6873.0
80/6873.0
81/6873.0
82/6873.0
83/6873.0
84/6873.0
85/6873.0
86/6873.0
87/6873.0
88/6873.0
89/6873.0
90/6873.0
91/6873.0
92/6873.0
93/6873.0
94/6873.0
95/6873.0
96/6873.0
97/6873.0
98/6873.0
99/6873.0
100/6873.0
101/6873.0
102/6873.0
103/6873.0
104/6873.0
105/6873.0
106/687

762/6873.0
763/6873.0
764/6873.0
765/6873.0
766/6873.0
767/6873.0
768/6873.0
769/6873.0
770/6873.0
771/6873.0
772/6873.0
773/6873.0
774/6873.0
775/6873.0
776/6873.0
777/6873.0
778/6873.0
779/6873.0
780/6873.0
781/6873.0
782/6873.0
783/6873.0
784/6873.0
785/6873.0
786/6873.0
787/6873.0
788/6873.0
789/6873.0
790/6873.0
791/6873.0
792/6873.0
793/6873.0
794/6873.0
795/6873.0
796/6873.0
797/6873.0
798/6873.0
799/6873.0
800/6873.0
801/6873.0
802/6873.0
803/6873.0
804/6873.0
805/6873.0
806/6873.0
807/6873.0
808/6873.0
809/6873.0
810/6873.0
811/6873.0
812/6873.0
813/6873.0
814/6873.0
815/6873.0
816/6873.0
817/6873.0
818/6873.0
819/6873.0
820/6873.0
821/6873.0
822/6873.0
823/6873.0
824/6873.0
825/6873.0
826/6873.0
827/6873.0
828/6873.0
829/6873.0
830/6873.0
831/6873.0
832/6873.0
833/6873.0
834/6873.0
835/6873.0
836/6873.0
837/6873.0
838/6873.0
839/6873.0
840/6873.0
841/6873.0
842/6873.0
843/6873.0
844/6873.0
845/6873.0
846/6873.0
847/6873.0
848/6873.0
849/6873.0
850/6873.0
851/6873.0
852/6873.0

1466/6873.0
1467/6873.0
1468/6873.0
1469/6873.0
1470/6873.0
1471/6873.0
1472/6873.0
1473/6873.0
1474/6873.0
1475/6873.0
1476/6873.0
1477/6873.0
1478/6873.0
1479/6873.0
1480/6873.0
1481/6873.0
1482/6873.0
1483/6873.0
1484/6873.0
1485/6873.0
1486/6873.0
1487/6873.0
1488/6873.0
1489/6873.0
1490/6873.0
1491/6873.0
1492/6873.0
1493/6873.0
1494/6873.0
1495/6873.0
1496/6873.0
1497/6873.0
1498/6873.0
1499/6873.0
1500/6873.0
1501/6873.0
1502/6873.0
1503/6873.0
1504/6873.0
1505/6873.0
1506/6873.0
1507/6873.0
1508/6873.0
1509/6873.0
1510/6873.0
1511/6873.0
1512/6873.0
1513/6873.0
1514/6873.0
1515/6873.0
1516/6873.0
1517/6873.0
1518/6873.0
1519/6873.0
1520/6873.0
1521/6873.0
1522/6873.0
1523/6873.0
1524/6873.0
1525/6873.0
1526/6873.0
1527/6873.0
1528/6873.0
1529/6873.0
1530/6873.0
1531/6873.0
1532/6873.0
1533/6873.0
1534/6873.0
1535/6873.0
1536/6873.0
1537/6873.0
1538/6873.0
1539/6873.0
1540/6873.0
1541/6873.0
1542/6873.0
1543/6873.0
1544/6873.0
1545/6873.0
1546/6873.0
1547/6873.0
1548/6873.0
1549

...

3523/6873.0
3524/6873.0
3525/6873.0
3526/6873.0
3527/6873.0
3528/6873.0
3529/6873.0
3530/6873.0
3531/6873.0
3532/6873.0
3533/6873.0
3534/6873.0
3535/6873.0
3536/6873.0
3537/6873.0
3538/6873.0
3539/6873.0
3540/6873.0
3541/6873.0
3542/6873.0
3543/6873.0
3544/6873.0
3545/6873.0
3546/6873.0
3547/6873.0
3548/6873.0
3549/6873.0
3550/6873.0
3551/6873.0
3552/6873.0
3553/6873.0
3554/6873.0
3555/6873.0
3556/6873.0
3557/6873.0
3558/6873.0
3559/6873.0
3560/6873.0
3561/6873.0
3562/6873.0
3563/6873.0
3564/6873.0
3565/6873.0
3566/6873.0
3567/6873.0
3568/6873.0
3569/6873.0
3570/6873.0
3571/6873.0
3572/6873.0
3573/6873.0
3574/6873.0
3575/6873.0
3576/6873.0
3577/6873.0
3578/6873.0
3579/6873.0
3580/6873.0
3581/6873.0
3582/6873.0
3583/6873.0
3584/6873.0
3585/6873.0
3586/6873.0
3587/6873.0
3588/6873.0
3589/6873.0
3590/6873.0
3591/6873.0
3592/6873.0
3593/6873.0
3594/6873.0
3595/6873.0
3596/6873.0
3597/6873.0
3598/6873.0
3599/6873.0
3600/6873.0
3601/6873.0
3602/6873.0
3603/6873.0
3604/6873.0
3605/6873.0
3606

4210/6873.0
4211/6873.0
4212/6873.0
4213/6873.0
4214/6873.0
4215/6873.0
4216/6873.0
4217/6873.0
4218/6873.0
4219/6873.0
4220/6873.0
4221/6873.0
4222/6873.0
4223/6873.0
4224/6873.0
4225/6873.0
4226/6873.0
4227/6873.0
4228/6873.0
4229/6873.0
4230/6873.0
4231/6873.0
4232/6873.0
4233/6873.0
4234/6873.0
4235/6873.0
4236/6873.0
4237/6873.0
4238/6873.0
4239/6873.0
4240/6873.0
4241/6873.0
4242/6873.0
4243/6873.0
4244/6873.0
4245/6873.0
4246/6873.0
4247/6873.0
4248/6873.0
4249/6873.0
4250/6873.0
4251/6873.0
4252/6873.0
4253/6873.0
4254/6873.0
4255/6873.0
4256/6873.0
4257/6873.0
4258/6873.0
4259/6873.0
4260/6873.0
4261/6873.0
4262/6873.0
4263/6873.0
4264/6873.0
4265/6873.0
4266/6873.0
4267/6873.0
4268/6873.0
4269/6873.0
4270/6873.0
4271/6873.0
4272/6873.0
4273/6873.0
4274/6873.0
4275/6873.0
4276/6873.0
4277/6873.0
4278/6873.0
4279/6873.0
4280/6873.0
4281/6873.0
4282/6873.0
4283/6873.0
4284/6873.0
4285/6873.0
4286/6873.0
4287/6873.0
4288/6873.0
4289/6873.0
4290/6873.0
4291/6873.0
4292/6873.0
4293

4894/6873.0
4895/6873.0
4896/6873.0
4897/6873.0
4898/6873.0
4899/6873.0
4900/6873.0
4901/6873.0
4902/6873.0
4903/6873.0
4904/6873.0
4905/6873.0
4906/6873.0
4907/6873.0
4908/6873.0
4909/6873.0
4910/6873.0
4911/6873.0
4912/6873.0
4913/6873.0
4914/6873.0
4915/6873.0
4916/6873.0
4917/6873.0
4918/6873.0
4919/6873.0
4920/6873.0
4921/6873.0
4922/6873.0
4923/6873.0
4924/6873.0
4925/6873.0
4926/6873.0
4927/6873.0
4928/6873.0
4929/6873.0
4930/6873.0
4931/6873.0
4932/6873.0
4933/6873.0
4934/6873.0
4935/6873.0
4936/6873.0
4937/6873.0
4938/6873.0
4939/6873.0
4940/6873.0
4941/6873.0
4942/6873.0
4943/6873.0
4944/6873.0
4945/6873.0
4946/6873.0
4947/6873.0
4948/6873.0
4949/6873.0
4950/6873.0
4951/6873.0
4952/6873.0
4953/6873.0
4954/6873.0
4955/6873.0
4956/6873.0
4957/6873.0
4958/6873.0
4959/6873.0
4960/6873.0
4961/6873.0
4962/6873.0
4963/6873.0
4964/6873.0
4965/6873.0
4966/6873.0
4967/6873.0
4968/6873.0
4969/6873.0
4970/6873.0
4971/6873.0
4972/6873.0
4973/6873.0
4974/6873.0
4975/6873.0
4976/6873.0
4977

5580/6873.0
5581/6873.0
5582/6873.0
5583/6873.0
5584/6873.0
5585/6873.0
5586/6873.0
5587/6873.0
5588/6873.0
5589/6873.0
5590/6873.0
5591/6873.0
5592/6873.0
5593/6873.0
5594/6873.0
5595/6873.0
5596/6873.0
5597/6873.0
5598/6873.0
5599/6873.0
5600/6873.0
5601/6873.0
5602/6873.0
5603/6873.0
5604/6873.0
5605/6873.0
5606/6873.0
5607/6873.0
5608/6873.0
5609/6873.0
5610/6873.0
5611/6873.0
5612/6873.0
5613/6873.0
5614/6873.0
5615/6873.0
5616/6873.0
5617/6873.0
5618/6873.0
5619/6873.0
5620/6873.0
5621/6873.0
5622/6873.0
5623/6873.0
5624/6873.0
5625/6873.0
5626/6873.0
5627/6873.0
5628/6873.0
5629/6873.0
5630/6873.0
5631/6873.0
5632/6873.0
5633/6873.0
5634/6873.0
5635/6873.0
5636/6873.0
5637/6873.0
5638/6873.0
5639/6873.0
5640/6873.0
5641/6873.0
5642/6873.0
5643/6873.0
5644/6873.0
5645/6873.0
5646/6873.0
5647/6873.0
5648/6873.0
5649/6873.0
5650/6873.0
5651/6873.0
5652/6873.0
5653/6873.0
5654/6873.0
5655/6873.0
5656/6873.0
5657/6873.0
5658/6873.0
5659/6873.0
5660/6873.0
5661/6873.0
5662/6873.0
5663

6270/6873.0
6271/6873.0
6272/6873.0
6273/6873.0
6274/6873.0
6275/6873.0
6276/6873.0
6277/6873.0
6278/6873.0
6279/6873.0
6280/6873.0
6281/6873.0
6282/6873.0
6283/6873.0
6284/6873.0
6285/6873.0
6286/6873.0
6287/6873.0
6288/6873.0
6289/6873.0
6290/6873.0
6291/6873.0
6292/6873.0
6293/6873.0
6294/6873.0
6295/6873.0
6296/6873.0
6297/6873.0
6298/6873.0
6299/6873.0
6300/6873.0
6301/6873.0
6302/6873.0
6303/6873.0
6304/6873.0
6305/6873.0
6306/6873.0
6307/6873.0
6308/6873.0
6309/6873.0
6310/6873.0
6311/6873.0
6312/6873.0
6313/6873.0
6314/6873.0
6315/6873.0
6316/6873.0
6317/6873.0
6318/6873.0
6319/6873.0
6320/6873.0
6321/6873.0
6322/6873.0
6323/6873.0
6324/6873.0
6325/6873.0
6326/6873.0
6327/6873.0
6328/6873.0
6329/6873.0
6330/6873.0
6331/6873.0
6332/6873.0
6333/6873.0
6334/6873.0
6335/6873.0
6336/6873.0
6337/6873.0
6338/6873.0
6339/6873.0
6340/6873.0
6341/6873.0
6342/6873.0
6343/6873.0
6344/6873.0
6345/6873.0
6346/6873.0
6347/6873.0
6348/6873.0
6349/6873.0
6350/6873.0
6351/6873.0
6352/6873.0
6353

In [38]:
len(bert_predicted)

np.mean(bert_predicted)

print(classification_report(test_y, bert_predicted))

print(epoch_loss)

6873

In [42]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(train_tokens_ids, train_y_tensor)

C:\Users\User\.conda\envs\Awesome\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC()

In [43]:
baseline_predicted = clf.predict(test_tokens_tensor)

In [44]:
print(classification_report(test_labels, baseline_predicted))

              precision    recall  f1-score   support

           0       0.13      0.08      0.10       688
           1       0.15      0.12      0.14       687
           2       0.13      0.07      0.10       687
           3       0.13      0.17      0.15       687
           4       0.10      0.30      0.15       687
           5       0.11      0.11      0.11       687
           6       0.12      0.07      0.09       687
           7       0.11      0.11      0.11       687
           8       0.15      0.06      0.08       687
           9       0.11      0.10      0.10       689

    accuracy                           0.12      6873
   macro avg       0.13      0.12      0.11      6873
weighted avg       0.13      0.12      0.11      6873



In [45]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer

classif = OneVsRestClassifier(estimator=SVC(random_state=1))
baseline_model = classif.fit(train_tokens_ids, train_y_tensor)
baseline_predicted = classif.predict(test_tokens_tensor)
print(classification_report(test_labels, baseline_predicted))

              precision    recall  f1-score   support

           0       0.18      0.17      0.17       688
           1       0.16      0.17      0.16       687
           2       0.15      0.10      0.12       687
           3       0.12      0.30      0.18       687
           4       0.19      0.14      0.16       687
           5       0.17      0.12      0.14       687
           6       0.17      0.16      0.16       687
           7       0.16      0.18      0.17       687
           8       0.17      0.12      0.14       687
           9       0.14      0.10      0.12       689

    accuracy                           0.16      6873
   macro avg       0.16      0.16      0.15      6873
weighted avg       0.16      0.16      0.15      6873

